In [34]:
import sys
import sdic
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from keras.layers import Dense, Dropout, Flatten, GaussianNoise, ZeroPadding2D, Input, concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential, Model, load_model
import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from pickle import load
from keras.preprocessing.image import ImageDataGenerator

In [15]:
def myGlobal0_to_1_Normalization(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [16]:
X = pd.read_csv("/workspace/DeepImageAI/expression_binary_autosome.csv")
Y = pd.read_csv("/workspace/DeepImageAI/input_meta_filtered_binray.csv")
Y = np.asarray(Y["Activity"])
X = (X.iloc[:, 1:].values).T
assert(X.shape[0] == Y.shape[0]) # Make sure we have the same number of samples

In [17]:
print(X.shape) # 550 samples and 33153 genes
print(Y.shape)

(550, 33153)
(550,)


In [18]:
P = 183
P * P
img_size = P
img_rows = P
img_cols = P

In [19]:
X1 = myGlobal0_to_1_Normalization(X) # MODE_ASIS
X1 = np.resize(X1, (550, P*P))
X1 = X1.reshape((550, P, P))
print(X1.shape) # Just like the code!
print(np.min(X1))
print(np.max(X1))

(550, 183, 183)
0.0
1.0


In [20]:
x_train = X
x_train = np.resize(x_train, (550, P*P))
x_train = x_train.reshape((550, P, P))

x_train_new_rand = np.zeros((x_train.shape[0], img_size, img_size))
#x_test_new_rand=np.zeros((x_test.shape[0],img_size,img_size))
ind=list(range(0,img_size*img_size))
np.random.seed(1234)
np.random.shuffle(ind)
cx=cy=0
dir=1
for i in range(0,img_size*img_size):
    dy=ind[i]//img_size
    dx=ind[i]%img_size
    x_train_new_rand[:,cy,cx]=x_train[:,dy,dx] 
#    x_test_new_rand[:,cy,cx]=x_test[:,dy,dx] 
    if dir==1: cx+=1
    else: cx-=1
    if cx==img_size:
        cy+=1
        cx-=1
        dir*=-1
    elif cx==-1:
        cy+=1
        cx+=1
        dir*=-1
    x_train_new=x_train_new_rand
    #x_test_new=x_test_new_rand
    
print(x_train.shape)
print(x_train_new.shape)
print(np.array_equal(x_train, x_train))
print(np.array_equal(x_train, x_train_new))
X2 = x_train_new # MODE_RAND

(550, 183, 183)
(550, 183, 183)
True
False


In [21]:
print(X.shape)
print(Y.shape)
X = myGlobal0_to_1_Normalization(X)

x_train = X
x_train = np.resize(x_train, (550, P*P))
x_train = x_train.reshape((550, P, P))

vic = sdic.sdic(sdic.SDIC_TYPE_SDIC)
vic.fit(x_train)
x_train_new = vic.transform(x_train)
X3 = x_train_new

(550, 33153)
(550,)


In [22]:
#
# Let's just train on the SDIC method for now!
#

1.0


In [24]:
x_train = X3
print(np.min(x_train))
print(np.max(x_train))
#x_train = myGlobal0_to_1_Normalization(x_train)
#print(np.min(x_train))
#print(np.max(x_train))

0.0
1.0


In [25]:
print(P)
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], 184, 184, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (184, 184, x_train.shape[3])
print(input_shape) # One channel!

183
(550, 184, 184)
(184, 184, 1)


In [26]:
y_train = Y
print(x_train.shape)
print(y_train.shape)

(550, 184, 184, 1)
(550,)


In [43]:
import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
#input_shape=np.shape(input_data_transformed)[1:4]

def createModel():
    input_img = Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(input_img)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x) 
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(x)    
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)    
    x = tf.keras.layers.Dropout(0.5)(x)    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)    
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='visualized_layer')(x)

    def auc(y_true, y_pred):
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)
    
    model = Model(inputs=input_img, outputs=output)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_crossentropy'])    
    return model

def newModel():
    model = Sequential()
    ks=(3,3)
    num_classes = 2
    model.add(Conv2D(32, kernel_size=(ks), activation='relu',input_shape=input_shape))
    model.add(Conv2D(64, ks, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) # Binary in NYG
    model.compile(loss=keras.losses.categorical_crossentropy if num_classes>2 else keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [51]:
model = createModel()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 184, 184, 1)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 182, 182, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 91, 91, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 91, 91, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 89, 89, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 44, 44, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 44, 44, 64)       

In [52]:
datagen = ImageDataGenerator()

In [68]:
#X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, stratify=y_train, test_size=0.25, shuffle=True)
#X_train = X_train.reshape((x_train.shape[0], x_train.shape[1], X_train.shape[2], 3))
#X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 3))

In [53]:
train_iterator = datagen.flow(x_train, y_train, batch_size=32, seed=17, shuffle=False)
#test_iterator = datagen.flow(X_test,  Y_test, batch_size=Y_test.shape[0], seed=17, shuffle=False)

In [54]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
print(y_train.shape)

{0: 4.583333333333333, 1: 0.5612244897959183}
(550,)


In [55]:
checkpoint_path_incep = "Binary_all_vs_control/cp_incep.ckpt"
checkpoint_dir_incep = os.path.dirname(checkpoint_path_incep)

from tensorflow.python.keras.callbacks import ModelCheckpoint
callbacks_list = [   
    ModelCheckpoint(filepath=checkpoint_path_incep, monitor='val_loss', save_best_only=True, save_weights_only=True,verbose=1, mode='min')
]

In [56]:
with tf.device('/gpu:0'):
    history = model.fit_generator(train_iterator,                           
                                  epochs=500,
                                  validation_data=train_iterator,
                                  #class_weight=d_class_weights,
                                  callbacks=callbacks_list)

Epoch 1/500
18/18 [==============================] - ETA: 0s - loss: 0.4813 - binary_crossentropy: 0.4813
Epoch 00001: val_loss improved from inf to 0.60078, saving model to Binary_all_vs_control/cp_incep.ckpt
18/18 [==============================] - 9s 515ms/step - loss: 0.4813 - binary_crossentropy: 0.4813 - val_loss: 0.6008 - val_binary_crossentropy: 0.6008
Epoch 2/500
18/18 [==============================] - ETA: 0s - loss: 0.4449 - binary_crossentropy: 0.4449
Epoch 00002: val_loss improved from 0.60078 to 0.50057, saving model to Binary_all_vs_control/cp_incep.ckpt
18/18 [==============================] - 9s 497ms/step - loss: 0.4449 - binary_crossentropy: 0.4449 - val_loss: 0.5006 - val_binary_crossentropy: 0.5006
Epoch 3/500
18/18 [==============================] - ETA: 0s - loss: 0.3866 - binary_crossentropy: 0.3866
Epoch 00003: val_loss improved from 0.50057 to 0.34268, saving model to Binary_all_vs_control/cp_incep.ckpt
18/18 [==============================] - 8s 467ms/step -

18/18 [==============================] - 8s 463ms/step - loss: 0.1130 - binary_crossentropy: 0.1130 - val_loss: 0.0937 - val_binary_crossentropy: 0.0937
Epoch 25/500
18/18 [==============================] - ETA: 0s - loss: 0.1045 - binary_crossentropy: 0.1045
Epoch 00025: val_loss improved from 0.09367 to 0.07834, saving model to Binary_all_vs_control/cp_incep.ckpt
18/18 [==============================] - 9s 480ms/step - loss: 0.1045 - binary_crossentropy: 0.1045 - val_loss: 0.0783 - val_binary_crossentropy: 0.0783
Epoch 26/500
18/18 [==============================] - ETA: 0s - loss: 0.0707 - binary_crossentropy: 0.0707
Epoch 00026: val_loss improved from 0.07834 to 0.04476, saving model to Binary_all_vs_control/cp_incep.ckpt
18/18 [==============================] - 8s 462ms/step - loss: 0.0707 - binary_crossentropy: 0.0707 - val_loss: 0.0448 - val_binary_crossentropy: 0.0448
Epoch 27/500
18/18 [==============================] - ETA: 0s - loss: 0.0889 - binary_crossentropy: 0.0889
Epoc

18/18 [==============================] - ETA: 0s - loss: 0.0536 - binary_crossentropy: 0.0536
Epoch 00049: val_loss did not improve from 0.00677
18/18 [==============================] - 6s 353ms/step - loss: 0.0536 - binary_crossentropy: 0.0536 - val_loss: 0.0353 - val_binary_crossentropy: 0.0353
Epoch 50/500
18/18 [==============================] - ETA: 0s - loss: 0.0317 - binary_crossentropy: 0.0317
Epoch 00050: val_loss did not improve from 0.00677
18/18 [==============================] - 6s 348ms/step - loss: 0.0317 - binary_crossentropy: 0.0317 - val_loss: 0.0127 - val_binary_crossentropy: 0.0127
Epoch 51/500
18/18 [==============================] - ETA: 0s - loss: 0.0139 - binary_crossentropy: 0.0139
Epoch 00051: val_loss did not improve from 0.00677
18/18 [==============================] - 6s 351ms/step - loss: 0.0139 - binary_crossentropy: 0.0139 - val_loss: 0.0093 - val_binary_crossentropy: 0.0093
Epoch 52/500
18/18 [==============================] - ETA: 0s - loss: 0.0100 - b

18/18 [==============================] - ETA: 0s - loss: 0.0031 - binary_crossentropy: 0.0031
Epoch 00073: val_loss did not improve from 0.00038
18/18 [==============================] - 6s 347ms/step - loss: 0.0031 - binary_crossentropy: 0.0031 - val_loss: 9.1203e-04 - val_binary_crossentropy: 9.1203e-04
Epoch 74/500
18/18 [==============================] - ETA: 0s - loss: 0.0101 - binary_crossentropy: 0.0101
Epoch 00074: val_loss did not improve from 0.00038
18/18 [==============================] - 6s 347ms/step - loss: 0.0101 - binary_crossentropy: 0.0101 - val_loss: 0.0053 - val_binary_crossentropy: 0.0053
Epoch 75/500
18/18 [==============================] - ETA: 0s - loss: 0.0060 - binary_crossentropy: 0.0060
Epoch 00075: val_loss did not improve from 0.00038
18/18 [==============================] - 6s 348ms/step - loss: 0.0060 - binary_crossentropy: 0.0060 - val_loss: 0.0042 - val_binary_crossentropy: 0.0042
Epoch 76/500
18/18 [==============================] - ETA: 0s - loss: 0.

18/18 [==============================] - ETA: 0s - loss: 2.5987e-04 - binary_crossentropy: 2.5987e-04
Epoch 00097: val_loss did not improve from 0.00007
18/18 [==============================] - 6s 350ms/step - loss: 2.5987e-04 - binary_crossentropy: 2.5987e-04 - val_loss: 8.6481e-05 - val_binary_crossentropy: 8.6481e-05
Epoch 98/500
18/18 [==============================] - ETA: 0s - loss: 5.7170e-04 - binary_crossentropy: 5.7170e-04
Epoch 00098: val_loss did not improve from 0.00007
18/18 [==============================] - 7s 362ms/step - loss: 5.7170e-04 - binary_crossentropy: 5.7170e-04 - val_loss: 2.6327e-04 - val_binary_crossentropy: 2.6327e-04
Epoch 99/500
18/18 [==============================] - ETA: 0s - loss: 0.0032 - binary_crossentropy: 0.0032 
Epoch 00099: val_loss did not improve from 0.00007
18/18 [==============================] - 6s 348ms/step - loss: 0.0032 - binary_crossentropy: 0.0032 - val_loss: 7.2565e-04 - val_binary_crossentropy: 7.2565e-04
Epoch 100/500
18/18 [==

...

18/18 [==============================] - ETA: 0s - loss: 0.0031 - binary_crossentropy: 0.0031
Epoch 00392: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 349ms/step - loss: 0.0031 - binary_crossentropy: 0.0031 - val_loss: 0.0015 - val_binary_crossentropy: 0.0015
Epoch 393/500
18/18 [==============================] - ETA: 0s - loss: 0.0011 - binary_crossentropy: 0.0011
Epoch 00393: val_loss did not improve from 0.00000
18/18 [==============================] - 7s 367ms/step - loss: 0.0011 - binary_crossentropy: 0.0011 - val_loss: 6.6004e-04 - val_binary_crossentropy: 6.6004e-04
Epoch 394/500
18/18 [==============================] - ETA: 0s - loss: 5.2338e-04 - binary_crossentropy: 5.2338e-04
Epoch 00394: val_loss did not improve from 0.00000
18/18 [==============================] - 7s 363ms/step - loss: 5.2338e-04 - binary_crossentropy: 5.2338e-04 - val_loss: 3.4900e-04 - val_binary_crossentropy: 3.4900e-04
Epoch 395/500
18/18 [=========================

Epoch 417/500
18/18 [==============================] - ETA: 0s - loss: 2.8391e-04 - binary_crossentropy: 2.8391e-04
Epoch 00417: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 352ms/step - loss: 2.8391e-04 - binary_crossentropy: 2.8391e-04 - val_loss: 2.2909e-04 - val_binary_crossentropy: 2.2909e-04
Epoch 418/500
18/18 [==============================] - ETA: 0s - loss: 2.4980e-04 - binary_crossentropy: 2.4980e-04
Epoch 00418: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 351ms/step - loss: 2.4980e-04 - binary_crossentropy: 2.4980e-04 - val_loss: 1.5655e-04 - val_binary_crossentropy: 1.5655e-04
Epoch 419/500
18/18 [==============================] - ETA: 0s - loss: 1.7761e-04 - binary_crossentropy: 1.7761e-04
Epoch 00419: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 352ms/step - loss: 1.7761e-04 - binary_crossentropy: 1.7761e-04 - val_loss: 1.1186e-04 - val_binary_crossentropy: 1.1

Epoch 442/500
18/18 [==============================] - ETA: 0s - loss: 2.4298e-05 - binary_crossentropy: 2.4298e-05
Epoch 00442: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 357ms/step - loss: 2.4298e-05 - binary_crossentropy: 2.4298e-05 - val_loss: 2.6059e-05 - val_binary_crossentropy: 2.6059e-05
Epoch 443/500
18/18 [==============================] - ETA: 0s - loss: 2.1712e-05 - binary_crossentropy: 2.1712e-05
Epoch 00443: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 354ms/step - loss: 2.1712e-05 - binary_crossentropy: 2.1712e-05 - val_loss: 2.4346e-05 - val_binary_crossentropy: 2.4346e-05
Epoch 444/500
18/18 [==============================] - ETA: 0s - loss: 2.2267e-05 - binary_crossentropy: 2.2267e-05
Epoch 00444: val_loss did not improve from 0.00000
18/18 [==============================] - 7s 362ms/step - loss: 2.2267e-05 - binary_crossentropy: 2.2267e-05 - val_loss: 2.2259e-05 - val_binary_crossentropy: 2.2

Epoch 467/500
18/18 [==============================] - ETA: 0s - loss: 7.4047e-06 - binary_crossentropy: 7.4047e-06
Epoch 00467: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 351ms/step - loss: 7.4047e-06 - binary_crossentropy: 7.4047e-06 - val_loss: 7.3519e-06 - val_binary_crossentropy: 7.3519e-06
Epoch 468/500
18/18 [==============================] - ETA: 0s - loss: 2.3142e-05 - binary_crossentropy: 2.3142e-05
Epoch 00468: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 348ms/step - loss: 2.3142e-05 - binary_crossentropy: 2.3142e-05 - val_loss: 7.1927e-06 - val_binary_crossentropy: 7.1927e-06
Epoch 469/500
18/18 [==============================] - ETA: 0s - loss: 6.3123e-06 - binary_crossentropy: 6.3123e-06
Epoch 00469: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 348ms/step - loss: 6.3123e-06 - binary_crossentropy: 6.3123e-06 - val_loss: 7.1227e-06 - val_binary_crossentropy: 7.1

Epoch 492/500
18/18 [==============================] - ETA: 0s - loss: 5.0718e-05 - binary_crossentropy: 5.0718e-05
Epoch 00492: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 351ms/step - loss: 5.0718e-05 - binary_crossentropy: 5.0718e-05 - val_loss: 3.1494e-05 - val_binary_crossentropy: 3.1494e-05
Epoch 493/500
18/18 [==============================] - ETA: 0s - loss: 2.6345e-05 - binary_crossentropy: 2.6345e-05
Epoch 00493: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 349ms/step - loss: 2.6345e-05 - binary_crossentropy: 2.6345e-05 - val_loss: 2.7603e-05 - val_binary_crossentropy: 2.7603e-05
Epoch 494/500
18/18 [==============================] - ETA: 0s - loss: 2.9814e-05 - binary_crossentropy: 2.9814e-05
Epoch 00494: val_loss did not improve from 0.00000
18/18 [==============================] - 6s 353ms/step - loss: 2.9814e-05 - binary_crossentropy: 2.9814e-05 - val_loss: 2.5892e-05 - val_binary_crossentropy: 2.5

In [58]:
print("Completed!")

Completed!


In [59]:
model_incep_loaded = createModel()

In [60]:
model_incep_loaded.load_weights(checkpoint_path_incep)

In [61]:
pred_train_inception_load = model_incep_loaded.predict_generator(train_iterator, steps=len(train_iterator), verbose=0)

Instructions for updating:
Please use Model.predict, which supports generators.


In [66]:
Y_train = y_train
print(Y_train.shape)
auc_train_external_stack_pos = roc_auc_score(Y_train, pred_train_inception_load)
print("auc_train_external_stack_pos: " + str(auc_train_external_stack_pos))
pred_train_external_stack_pos = np.where(pred_train_inception_load > 0.5, 1, 0)
tn, fp, fn, tp = confusion_matrix(Y_train, pred_train_external_stack_pos).ravel()
specificity_train_external_stack_pos = tn / (tn + fp)
sensitivity_train_external_stack_pos = tp / (tp + fn)
NPV_train_external_stack_pos = tn / (tn + fn)
PPV_train_external_stack_pos = tp / (tp + fp)
Accuracy_train_external_stack_pos = balanced_accuracy_score(Y_train, pred_train_external_stack_pos)
MCC_train_external_stack_pos = matthews_corrcoef(Y_train, pred_train_external_stack_pos)
print("specificity_train_external_stack_pos: " + str(specificity_train_external_stack_pos))
print("sensitivity_train_external_stack_pos: " + str(sensitivity_train_external_stack_pos))
print("NPV_train_external_stack_pos: " + str(NPV_train_external_stack_pos))
print("PPV_train_external_stack_pos: " + str(PPV_train_external_stack_pos))
print("Accuracy_train_external_stack_pos: " + str(Accuracy_train_external_stack_pos))
print("MCC_train_external_stack_pos: " + str(MCC_train_external_stack_pos))

from sklearn.metrics import f1_score
print("F1 SCORE train= "+str(f1_score(Y_train, pred_train_external_stack_pos, average='macro')))
print(classification_report(Y_train, pred_train_external_stack_pos))

(550,)
auc_train_external_stack_pos: 1.0
specificity_train_external_stack_pos: 1.0
sensitivity_train_external_stack_pos: 1.0
NPV_train_external_stack_pos: 1.0
PPV_train_external_stack_pos: 1.0
Accuracy_train_external_stack_pos: 1.0
MCC_train_external_stack_pos: 1.0
F1 SCORE train= 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00       490

    accuracy                           1.00       550
   macro avg       1.00      1.00      1.00       550
weighted avg       1.00      1.00      1.00       550



550